Тут я использую нейронку EIDM. Ее описание встречается только тут:

https://link.springer.com/article/10.1007/s11227-023-05197-0

Сейчас она распознает только 4 класса веб атак. В теории она сможет хорошо распозновать 15 класаов.

In [ ]:
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score,classification_report
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Embedding
from tensorflow.keras.layers import LSTM, Conv1D,Conv2D, MaxPooling1D, Flatten,Bidirectional

from tensorflow.keras.optimizers import RMSprop, SGD
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import MaxAbsScaler, normalize, LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
# MODEL_PATH = './models/EIDM_model.keras'
MODEL_PATH = "./models/EIDM_2.keras"

In [ ]:
from google.colab import drive
from tqdm import tqdm
import os
drive.mount('/content/drive')
#папка для сохранения чего-то
shared_folder_path = '/content/drive/MyDrive/dudos/'

Mounted at /content/drive


In [ ]:
def save_df( path: str, name: str):
  # Убедитесь, что папка существует
  os.makedirs(path, exist_ok=True)

  # Определите путь для сохранения файла
  file_path = os.path.join(path, name)

  # Укажите размер чанка
  chunksize = 10000

  # Откройте файл для записи
  with open(file_path, 'w') as f:
      # Запишите заголовок вручную
      df.iloc[:0].to_csv(f, index=False)

      # Создайте tqdm-объект для отслеживания прогресса
      with tqdm(total=len(df), desc="Saving to CSV", unit="row") as pbar:
          # Итерация по чанкам
          for i in range(0, len(df), chunksize):
              df_chunk = df.iloc[i:i+chunksize]
              df_chunk.to_csv(f, header=False, index=False)
              pbar.update(len(df_chunk))

  # Отображение количества значений в столбце 'Label'
  print(df['Label'].value_counts())

In [ ]:
import requests, zipfile, io
from tqdm import tqdm

# Работа с датосетом

In [ ]:
#data_path = "./web_attacks_balanced.csv"
data_path = shared_folder_path + "web_attacks_balanced.csv"
#read dataset
data= pd.read_csv(data_path).replace([np.inf, -np.inf], np.nan).dropna(how="any")
data = data.iloc[:, :-7]
data.shape

(1162213, 78)

In [ ]:
# Вывод первых 5 строк набора данных
print(data.head())

# Вывод информации о наборе данных
print(data.info())

   Destination Port  Flow Duration  Total Fwd Packets  Total Backward Packets  \
0           54865.0            3.0                2.0                     0.0   
1           55054.0          109.0                1.0                     1.0   
2           55055.0           52.0                1.0                     1.0   
3           46236.0           34.0                1.0                     1.0   
4           54863.0            3.0                2.0                     0.0   

   Total Length of Fwd Packets  Total Length of Bwd Packets  \
0                         12.0                          0.0   
1                          6.0                          6.0   
2                          6.0                          6.0   
3                          6.0                          6.0   
4                         12.0                          0.0   

   Fwd Packet Length Max  Fwd Packet Length Min  Fwd Packet Length Mean  \
0                    6.0                    6.0            

In [ ]:
filtered_data = data.loc[data['Label'] == 'BENIGN']
_=filtered_data.pop('Label')

In [ ]:
#split dataset to train and test

labels = data.pop('Label')
# Вывод уникальных значений меток
unique_labels = labels.unique()
print("Unique Labels:")
print(unique_labels)
#print(labels)
pdtrain_data, pdtest_data, pdtrain_Label, pdtest_label = train_test_split(data,labels, test_size=0.2,shuffle=True,stratify=labels)

Unique Labels:
['BENIGN' 'DDoS' 'PortScan' 'Bot' 'Infiltration'
 'Web Attack � Brute Force' 'Web Attack � XSS'
 'Web Attack � Sql Injection']


In [ ]:
pdtrain_Label

679161     BENIGN
834293     BENIGN
63657        DDoS
862377     BENIGN
963555     BENIGN
            ...  
1021898    BENIGN
1025017    BENIGN
997625     BENIGN
224816     BENIGN
1113434    BENIGN
Name: Label, Length: 929770, dtype: object

In [ ]:
#convert data from pd to numpy arrary
train_data = pdtrain_data.to_numpy()
test_data = pdtest_data.to_numpy()

filtered_data=filtered_data.to_numpy()


train_data = train_data.astype('float32')
test_data = test_data.astype('float32')
filtered_data= filtered_data.astype('float32')


#normalize data
train_data= normalize(train_data, axis=1, norm='l1')
test_data= normalize(test_data, axis=1, norm='l1')
filtered_data = normalize(filtered_data, axis=1, norm='l1')

In [ ]:
train_data[0]

array([4.27737348e-02, 3.27286660e-03, 2.19165177e-05, 7.30550573e-06,
       3.14136734e-04, 4.38330353e-05, 2.26470685e-04, 4.38330353e-05,
       1.04712250e-04, 1.05445892e-04, 4.38330353e-05, 4.38330353e-05,
       4.38330353e-05, 0.00000000e+00, 7.99039662e-01, 6.52277321e-02,
       1.09095545e-03, 1.35910558e-03, 2.62267655e-03, 2.92220229e-05,
       2.65189866e-03, 1.32594933e-03, 1.83384924e-03, 2.62267655e-03,
       2.92220229e-05, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 4.38330346e-04, 1.46110120e-04,
       4.89207990e-02, 1.63069330e-02, 4.38330353e-05, 2.26470685e-04,
       8.03605653e-05, 8.16780375e-05, 9.13188211e-04, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 7.30550573e-06,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.00450707e-04, 1.04712250e-04, 4.38330353e-05, 0.00000000e+00,
      

In [ ]:
#reshape data to 3D (потому что в основе и первым слоем идет сверточная нейронная сеть (она работает с фото))
train_data=train_data.reshape(train_data.shape[0],train_data.shape[1])
test_data=test_data.reshape(test_data.shape[0],test_data.shape[1])
filtered_data = filtered_data.reshape(filtered_data.shape[0], filtered_data.shape[1])

In [ ]:

#feature selection
test = SelectKBest(score_func=f_classif, k=77)
fit = test.fit(train_data, pdtrain_Label)
np.set_printoptions(precision=3)
# print(fit.scores_)

train_data = fit.transform(train_data)
# features = fit.transform(train_data)

# Summarize selected features

# train_data = features
print(test_data.shape)
test_data = fit.transform(test_data)

filtered_data = fit.transform(filtered_data)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [31 33 55 56 57 58 59 60] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


(232443, 77)


In [ ]:
# подготовка и нормализация данных, что бы на этом можно было обучать нейронную сеть

label_encoder = LabelEncoder()
#convert string labels to int values to be able to classify them
trainVec = label_encoder.fit_transform(pdtrain_Label)
testVec = label_encoder.fit_transform(pdtest_label)


#categorize labels
train_labels = keras.utils.to_categorical(trainVec)
test_labels = keras.utils.to_categorical(testVec)

In [ ]:
# reshape data to 3D (потому что в основе и первым слоем идет сверточная нейронная сеть (она работает с фото))
train_data=train_data.reshape(train_data.shape[0],train_data.shape[1],1)
test_data=test_data.reshape(test_data.shape[0],test_data.shape[1],1)
filtered_data = filtered_data.reshape(filtered_data.shape[0], filtered_data.shape[1],1)
print(train_data.shape)

(929770, 77, 1)


In [ ]:
train_data.shape

(929770, 77, 1)

# Настройка модели

In [ ]:
# prepare  Model
model = Sequential()

initializer = keras.initializers.LecunUniform()

model.add(Dense(77,kernel_initializer=initializer,activation='relu'))
# model.add(Conv1D(100, 20, activation='relu'))
model.add(Conv1D(60, 10, activation='relu'))
model.add(MaxPooling1D())
model.add(Dropout(0.2))
# Flatten the results to one dimension for passing into our next layer
model.add(Flatten())
model.add(Dense(120,kernel_initializer=initializer,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(80,kernel_initializer=initializer,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(40,kernel_initializer=initializer,activation='relu'))
model.add(Dropout(0.2))
# model.add(Dense(60,kernel_initializer=initializer,activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(60,kernel_initializer=initializer,activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(40,kernel_initializer=initializer,activation='relu'))
# model.add(Dropout(0.2))
initializer = keras.initializers.GlorotNormal()

model.add(Dense(8, activation='sigmoid',kernel_initializer=initializer))
# In[33]:


#opt = RMSprop(lr = 0.008, momentum=0.9)
opt = keras.optimizers.Adam(learning_rate=0.0005)
# opt = keras.optimizers.Adam(learning_rate=0.005)
# opt = SGD(lr=0.05,momentum=0.9)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])



In [ ]:

print(train_data.shape)
print(train_labels.shape)

print(test_data.shape)
print(test_labels.shape)
model = keras.models.load_model('/content/drive/MyDrive/dudos/models/EIDM_8_4_classes.keras')
history = model.fit(train_data, train_labels,
                    batch_size=3000,
                    epochs=100,
                    validation_data=(test_data, test_labels))



(929770, 77, 1)
(929770, 8)
(232443, 77, 1)
(232443, 8)
Epoch 1/100
310/310 [==============================] - 655s 2s/step - loss: 0.1023 - accuracy: 0.9574 - val_loss: 0.0871 - val_accuracy: 0.9600
Epoch 2/100
310/310 [==============================] - 610s 2s/step - loss: 0.0972 - accuracy: 0.9588 - val_loss: 0.0840 - val_accuracy: 0.9623
Epoch 3/100
310/310 [==============================] - 645s 2s/step - loss: 0.0925 - accuracy: 0.9603 - val_loss: 0.0795 - val_accuracy: 0.9623
Epoch 4/100
310/310 [==============================] - 649s 2s/step - loss: 0.0876 - accuracy: 0.9622 - val_loss: 0.0741 - val_accuracy: 0.9663
Epoch 5/100
310/310 [==============================] - 655s 2s/step - loss: 0.0825 - accuracy: 0.9640 - val_loss: 0.0677 - val_accuracy: 0.9684
Epoch 6/100
310/310 [==============================] - 610s 2s/step - loss: 0.0769 - accuracy: 0.9665 - val_loss: 0.0599 - val_accuracy: 0.9746
Epoch 7/100
310/310 [==============================] - 606s 2s/step - loss: 0.06

KeyboardInterrupt: 

In [ ]:
# model.save('/content/drive/MyDrive/dudos/models/EIDM_8_4_classes.keras')
model = keras.models.load_model('/content/drive/MyDrive/dudos/models/EIDM_8_4_classes.keras')
model.save('/content/drive/MyDrive/dudos/models/EIDM_8_4_classes.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Проверка результатов

In [ ]:
test_labels

In [ ]:
model.save(MODEL_PATH)

In [ ]:
# model = keras.models.load_model('/content/drive/MyDrive/dudos/models/EIDM_8_4_classes.keras')
model = keras.models.load_model('/content/drive/MyDrive/dudos/models/EIDM_8_4_classes.h5')

In [ ]:
# чем меньше loss и больше accuracy тем лучше
# ниже мы проверяем нейронную сеть

score = model.evaluate(test_data, test_labels, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


predict = model.predict(test_data,verbose=1)
print(predict)
test_labels


Test loss: 0.014822660014033318
Test accuracy: 0.9956075549125671
7264/7264 [==============================] - 81s 11ms/step
[[9.392e-02 2.780e-12 9.999e-01 ... 1.602e-08 3.356e-08 2.255e-08]
 [9.673e-01 1.598e-03 1.539e-02 ... 1.394e-03 5.352e-04 7.187e-04]
 [9.955e-01 1.066e-03 9.895e-06 ... 3.105e-05 1.159e-05 1.119e-05]
 ...
 [1.000e+00 7.876e-03 5.616e-05 ... 3.089e-07 1.135e-07 4.369e-08]
 [9.984e-01 2.697e-06 1.866e-06 ... 8.685e-07 3.144e-07 3.698e-07]
 [6.540e-01 1.495e-14 4.069e-18 ... 1.601e-18 1.533e-17 1.657e-17]]


array([[0., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
b = np.zeros_like(predict)
b[np.arange(len(predict)), predict.argmax(1)] = 1

# print('Точность равна =',(test_labels == b).sum()/(len(b)*4))
count = 0
a = b.shape
for i in range(a[0]):
    if (b[i] == test_labels[i]).all()  :
        count+=1
# ! грубая это НЕОФИЦИАЛЬНЫЙ ТЕРМИН.
print('Грубая точность', count/a[0])

In [ ]:
result = model.predict(filtered_data[:3000])
result

In [ ]:
len(result)
count = 0
for i in range(len(result[:3000])):
  max_v = np.where(result[i] == np.amax(result[i]))[0][0]
  if max_v != 0:
    count += 1
    print(f"Обнаружена атака формата {unique_labels[max_v]}")
if count == 0:
  print(f"Атак не обноруженно")
  count = 0